In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import firedrake
from firedrake import inner, grad, dx, sqrt, perp, min_value, max_value

In [ ]:
coarse_mesh = firedrake.Mesh('cv.msh')
mesh_hierarchy = firedrake.MeshHierarchy(coarse_mesh, 2)
mesh = mesh_hierarchy[-1]
Q = firedrake.FunctionSpace(mesh, family='CG', degree=1)

### A bit of distribution theory

Let $K$ be some subset of a domain $\Omega$ in Euclidean space, and let $\chi_K$ be the characteristic function of $K$:

$$\chi_K(x) = \begin{cases}1 & x \in K \\ 0 & x \notin K \end{cases}$$

**The thing I want to show you**: Let $v$ be a vector field in $\Omega$ and let $n$ be the unit outward normal vector to $K$.
Then

$$\int_\Omega\nabla\chi_K\cdot v\hspace{2pt}dx = -\int_{\partial K}v\cdot n\hspace{2pt}ds.$$

This is a little weird because $\chi_K$ is discontinuous -- how can it have a gradient?

First let's try a numerical example.
The whole domain $\Omega$ will be the unit square, and the subset $K$ will be the circle of radius 1/4 about the point $(1/2, 1/2)$.
We'll approximate the characteristic function of $K$ by a function $\chi_\delta$ that's piecewise linear within each triangle of a mesh.

In [ ]:
x = firedrake.SpatialCoordinate(mesh)

ξ = firedrake.as_vector((1/2, 1/2))
r = 1/4

δ = 0.025
expr = min_value(1, max_value(0, 1/δ - sqrt(inner(x - ξ, x - ξ))/(δ * r)))
χ_δ = firedrake.interpolate(expr, Q)

In [ ]:
firedrake.plot(χ_δ, cmap='magma_r', plot3d=True)

Next we'll compute the gradient.
It's a bit messy but you can see the clustering of triangles around the boundary of $K$.

In [ ]:
Z = firedrake.FunctionSpace(mesh, family='DG', degree=0)
z = firedrake.project(sqrt(inner(grad(χ_δ), grad(χ_δ))), Z)

In [ ]:
firedrake.plot(z, cmap='magma_r', plot3d=True)

Next we'll create the vector field

$$v = x - \xi$$

where $\xi = (1/2, 1/2)$ is the center of $K$, and numerically evaluate the integral

$$\int_\Omega\nabla\chi_\delta\cdot v\hspace{2pt}dx.$$

In [ ]:
V = firedrake.VectorFunctionSpace(mesh, family='CG', degree=1)
expr = x - ξ
v = firedrake.interpolate(expr, V)

In [ ]:
J = inner(grad(χ_δ), v) * dx
print(firedrake.assemble(J))

But we can calculate analytically that

$$\int_{\partial K}v\cdot n\hspace{2pt}ds = \pi / 8 \approx 0.39$$

which is close enough.

### More generally

Define the *signed distance function* $\sigma_K$ to be

$$\sigma_K(x) = \begin{cases}+\text{dist}(x, K) & x \not in K \\ -\text{dist}(x, K^c) & x \in K\end{cases}$$

For example, the signed distance function to a circle is a cone.
We can come up with a sequence $\chi_\delta$ of approximations to the characteristic function of $K$ as

$$\chi_\delta(x) = \max\{0, \min\{1, (1 - \delta^{-1}\sigma_K(x))/2\}\}.$$

This function is 0 further than $\delta$ away from $K$, $1$ further inside $K$ than $\delta$, and roughly linear along the normal vector in between.
Moreover $\chi_\delta$ converges to $\chi_K$ in the 1-norm as $\delta \to 0$.

The gradient of $\chi_\delta$ has norm $\delta^{-1}$ in a strip of width $\delta$ around the boundary of $K$ and is pointed along the unit normal vector $n$.
So for any nice vector field $v$,

$$\int_{\Omega}\nabla\chi_\delta\cdot v\hspace{2pt}dx = -\frac{1}{\delta}\int_{\{x: \text{dist}(x, \partial K) < \delta/2\}}v\cdot n\hspace{2pt}dx\ldots$$

where $n$ is some extension of the unit normal vector field to the region around $K$.
In the limit as $\delta\to 0$ the volume integral on the right-hand side approaches the surface integral

$$\ldots \to \int_{\partial K}v\cdot n\hspace{2pt}ds.$$

**Despite the fact that the gradient of $\chi_K$ is not a function, it still acts on functions in a coherent and well-defined way.**
This is the core idea of distribution theory.
If you've studied measure theory, it's also fair to say that

$$\nabla\chi_K = -n\cdot\Sigma_{\partial K}$$

where $\Sigma_{\partial K}$ is the surface (or Hausdorff) measure concentrated on the surface $\partial K$.

### Conservation laws

A conservation law describes the way that a field $q$ evolves in time from a *flux function* $F(q, \nabla q, \ldots)$.
For any control volume $K$ within $\Omega$, the rate of change of $q$ within $K$ plus the fluxes of $q$ across $\partial K$ is equal to the sources:

$$\frac{d}{dt}\int_Kq\hspace{2pt}dx + \int_{\partial K}F(q, \nabla q, \ldots)\cdot n\hspace{2pt}ds = \int_Kf\hspace{2pt}dx.$$

Examples:

* Mass density $\rho$, flux = $\rho u$ where $u$ is velocity.
* Momentum $\rho u$, flux = $\rho u \otimes u - \sigma$ where $\sigma$ = stress tensor.
* Thermal energy $\rho c_pT$ where $c_p$ = specific heat capcity, flux = $\rho c_pTu - k\nabla T$ where $k$ is conductivity coefficient.

Using the divergence theorem, you can derive a PDE from this *provided that all the fields are nice*.
But they aren't!
Instead, let's derive:

### Weak forms

Using the identities from before about characteristic functions, we can rewrite the conservation law as

$$\int_\Omega\left(\frac{\partial q}{\partial t}\chi_K - F(q, \nabla q, \ldots)\cdot\nabla\chi_K\right)dx = \int_\Omega f\chi_K\hspace{2pt}dx.$$

This equation is additive in $\chi_K$, i.e. if we have several sets $K_1, \ldots, K_m$ and a function $\phi$ such that

$$\phi = \alpha_1\chi_{K_1} + \ldots \alpha_m\chi_{K_m},$$

then we can rewrite the last equation as

$$\int_\Omega\left(\frac{\partial q}{\partial t}\phi - F(q, \nabla q, \ldots)\cdot\nabla\phi\right)dx = \int_\Omega f\phi\hspace{2pt}dx.$$

Functions $\phi$ that can be written as a finite linear combination of characteristic functions are called *simple*.
Now we can take limits.
If $\phi$ is an arbitrary function in $L^2$ such that $\nabla\phi$ is also in $L^2$, there is a sequence of simple functions $\{\phi_n\}$ that converge (weakly) to $\phi$.
Consequently, the last equation also holds for any $\phi$ within some function space.
This is called the **weak form** of the PDE and it is completely equivalent to the conservation law.
**The weak form replaces arbitrary control volumes with arbitrary test functions.**

### Example

The conservation form of steady-state heat conduction is

$$-\int_{\partial K} k\nabla T\cdot n\hspace{2pt}ds = \int_Kf\hspace{2pt}dx$$

for all control volumes $K$, where $f$ are heat sources.
To get the weak form, surface integral $\to$ volume integrals, flip a sign, and multiply by the gradient of a test function:

$$\int_\Omega k\nabla T\cdot\nabla\phi\hspace{2pt}dx = \int_\Omega f\phi\hspace{2pt}dx.$$

Something that becomes really obvious now is that the weak form is the same if we flip the roles of $T$ and $\phi$.
Moreover, if we take $\phi = T$, the left-hand side is always positive.
In other words, the weak form of the Poisson equation is symmetric and positive-definite.

### In general

In a lot of finite element or PDE textbooks, you'll see references to a **bilinear form**.
The idea is that weak forms are linear in both the field you're solving for and the test function.
This type of problem can then be written in an abstract form as

$$\mathscr{A}(q, \phi) = \langle f, \phi\rangle$$

where $\mathscr{A}$ is the weak form of the PDE and $\langle\cdot,\cdot\rangle$ is the duality pairing.
For time-dependent problems, this instead becomes

$$\left\langle\frac{\partial q}{\partial t}, \phi\right\rangle + \mathscr{A}(q, \phi) = \langle f, \phi\rangle.$$

For example, the bilinear form for the Laplace equation is

$$\mathscr{A}(T, \phi) = \int_\Omega k\nabla T\cdot\nabla\phi\hspace{2pt}dx.$$

### Galerkin's method

The solutions of a conservation law or a weak-form PDE live in an infinite-dimensional vector space.
(Sobolev spaces, oooh scary!)
**Galerkin's method** is a way to find a good approximating function within a finite-dimensional vector space.
For now we'll focus on steady-state type problems but you can probably guess how it goes for time-dependent ones.

First (and this part is on you!), pick a set of *trial functions* $\{\psi_1, \ldots, \psi_m\}$ and let $Q_m$ be the linear span of these basis functions.
We will seek an approximation

$$\hat q = q_1\psi_1 + \ldots q_m\psi_m$$

to the true solution $q$ of some weak form PDE.
To decide on a unique value of $\hat q$, we pick a set of *test functions* $\{\phi_1, \ldots, \phi_m\}$, and require that

$$\mathscr{A}(\hat q, \phi_i) = \langle f, \phi_i\rangle$$

for all $i$.
If we write out the expansion for $\hat q$, we then get that

$$\sum_j\mathscr{A}(\psi_j, \phi_i)q_j = \langle f, \phi_i\rangle$$

for all $i$.
But this looks like a matrix equation!
Define the *stiffness matrix* and *load vector*

$$A_{ij} = \mathscr{A}(\psi_j, \phi_i), \quad F_i = \langle f, \phi_i\rangle.$$

Then the coefficients $\mathbf{q}$ of $\hat q$ are the solution of the linear system

$$\mathbf{A}\mathbf{q} = \mathbf{F}.$$

### Trial and test functions

There are lots of ways to pick trial and test functions.
With spectral methods, you use trigonometric polynomials or, more generally, any orthonormal sequence.
With the finite element method, you use piecewise polynomial functions in each triangle of a mesh.
They're all Galerkin's method though.

For problems like the Laplace equation, the bilinear form is symmetric:

$$\mathscr{A}(\psi, \phi) = \mathscr{A}(\phi, \psi).$$

**For symmetric bilinear forms, using the same trial and test functions gives a symmetric matrix.**
For advection-type equations, there's no symmetry to preserve, and using a different set of test functions can be very advantageous.